## Tensor backbone

In [57]:
from dataclasses import dataclass
from typing import Union, List, Callable, Optional, Tuple, Literal

import numpy as np

Scalar = Union[int, float]

Data = Union[Scalar, list, np.ndarray, "Tensor"]


@dataclass(frozen=True)
class Leaf:
    value: "Tensor"
    grad_fn: Callable[[np.ndarray], np.ndarray]


class Tensor:
    def __init__(
        self,
        data: Data,
        requires_grad: bool = False,
        dependencies: Optional[List[Leaf]] = None,
        dtype=np.float32
    ):
        self._data = Tensor.build_ndarray(data, dtype)
        self.dtype = dtype

        self.requires_grad = requires_grad
        self.dependencies = dependencies or []

        self.grad = np.zeros_like(self._data) if requires_grad else None

    @property
    def data(self) -> np.ndarray:
        return self._data

    @data.setter
    def data(self, data: Data):
        self._data = Tensor.build_ndarray(data, self.dtype)
        if self.requires_grad:
            self.zero_grad()

    @property
    def size(self) -> int:
        return self.data.size

    @property
    def shape(self) -> Tuple[int, ...]:
        return self.data.shape

    @property
    def ndim(self) -> int:
        return self.data.ndim

    @staticmethod
    def build_ndarray(data: Data, dtype=np.float32) -> np.ndarray:
        if isinstance(data, Tensor):
            return np.array(data.data, dtype=dtype)
        if isinstance(data, np.ndarray):
            return data.astype(dtype)
        return np.array(data, dtype=dtype)
    
    @staticmethod
    def data_gate(data: Data) -> "Tensor":
        if isinstance(data, Tensor):
            return data
        return Tensor(data)

    def __repr__(self):
        return f"Tensor({self.data}, requires_grad={self.requires_grad}, shape={self.shape})"

    def zero_grad(self):
        if self.grad is None:
            self.grad = np.zeros_like(self._data)
        else:
            self.grad.fill(0.0)

    def backward(self, grad: Optional[np.ndarray] = None) -> None:
        if not self.requires_grad:
            raise RuntimeError(
                "Cannot call backward() on a tensor that does not require gradients. "
                "If you need gradients, ensure that requires_grad=True when creating the tensor."
            )

        if grad is None:
            if self.shape == ():
                grad = np.array(1.0)
            else:
                raise ValueError("Grad must be provided if tensor has shape")
            
        self.grad = self.grad + grad

        for dependency in self.dependencies:
            backward_grad = dependency.grad_fn(grad)
            dependency.value.backward(backward_grad)

    def transpose(self, axes: Tuple[int, ...] = None) -> "Tensor":
        # Perform the transpose operation
        output = np.transpose(self.data, axes=axes)

        # Handle dependencies for autograd
        dependencies: List[Leaf] = []

        if self.requires_grad:
            def _bkwd(grad: np.ndarray) -> np.ndarray:
                # Compute the inverse permutation of axes for the backward function
                if axes is None:
                    # Implicitly reverses transpose
                    return np.transpose(grad)  
                else:
                    # Compute the inverse permutation of axes
                    inv_axes = tuple(np.argsort(axes))
                    # Transpose the gradient back using the inverse permutation
                    return np.transpose(grad, axes=inv_axes)

            dependencies.append(
                Leaf(value=self, grad_fn=_bkwd)
            )

        # Return the new tensor with the transposed data
        return Tensor(
            output,
            requires_grad=self.requires_grad,
            dependencies=dependencies
        )

    @property
    def T(self):
        return self.transpose()

    @staticmethod
    def matmul(a: "Tensor", b: "Tensor") -> "Tensor":
        r"""
        Static method to perform matrix multiplication of two tensors.

        Args:
            a (Tensor): First matrix.
            b (Tensor): Second matrix.

        Returns:
            Tensor: Resulting tensor with tracked dependencies.
        """
        
        output = a.data @ b.data
        requires_grad = a.requires_grad or b.requires_grad
        dependencies = []

        if a.requires_grad:
            def _bkwd_a(grad: np.ndarray) -> np.ndarray:
                if b.ndim > 1:
                    return grad @ b.data.swapaxes(-1, -2)
                return np.outer(grad, b.data.T).squeeze()
            
            dependencies.append(
                Leaf(
                    value=a,
                    grad_fn=_bkwd_a
                )
            )

        if b.requires_grad:
            def _bkwd_b(grad: np.ndarray) -> np.ndarray:
                if a.ndim > 1:
                    return a.data.swapaxes(-1, -2) @ grad
                return np.outer(a.data.T, grad).squeeze()
            
            dependencies.append(
                Leaf(
                    value=b,
                    grad_fn=_bkwd_b
                )
            )

        return Tensor(output, requires_grad, dependencies)
    
    def dot(self, other: Data) -> "Tensor":
        return Tensor.matmul(self, Tensor.data_gate(other))
    
    def __matmul__(self, other: Data) -> "Tensor":
        return self.dot(other)

    # @staticmethod
    def bkwd_broadcast(tensor: "Tensor"):
        def _bkwd(grad: np.ndarray) -> np.ndarray:
            if tensor.ndim == 0:
                return np.sum(grad)

            if grad.ndim == 0:
                return grad

            ndim_added = max(0, grad.ndim - tensor.ndim)

            if ndim_added > 0:
                grad = grad.sum(axis=tuple(range(ndim_added)), keepdims=False)

            reduce_axes = tuple(
                dim for dim in range(tensor.ndim)
                if tensor.shape[dim] == 1 and grad.shape[dim] > 1
            )

            if reduce_axes:
                grad = grad.sum(axis=reduce_axes, keepdims=True)

            if grad.shape != tensor.shape:
                grad = grad.reshape(tensor.shape)

            return grad

        return _bkwd

    @staticmethod
    def add(a: "Tensor", b: "Tensor") -> "Tensor":
        output = a.data + b.data

        requires_grad = a.requires_grad or b.requires_grad

        dependencies = []

        if a.requires_grad:
            dependencies.append(
                Leaf(
                    value=a,
                    grad_fn=Tensor.bkwd_broadcast(a)
                )
            )

        if b.requires_grad:
            dependencies.append(
                Leaf(
                    value=b,
                    grad_fn=Tensor.bkwd_broadcast(b)
                )
            )

        return Tensor(output, requires_grad, dependencies)

    def __add__(self, other: Data) -> "Tensor":
        return Tensor.add(self, Tensor.data_gate(other))
    
    def __radd__(self, other: Data) -> "Tensor":
        return Tensor.add(Tensor.data_gate(other), self)
    
    def __iadd__(self, other: Data) -> "Tensor":
        self.data = self.data + Tensor.build_ndarray(other)
        return self
    
    def __neg__(self) -> "Tensor":
        output = -self.data
        dependencies = []

        if self.requires_grad:
            dependencies.append(
                Leaf(value=self, grad_fn=lambda grad: -grad)
            )

        return Tensor(output, self.requires_grad, dependencies)
    
    def __sub__(self, other: Data) -> "Tensor":
        return self + (-Tensor.data_gate(other))

    def __rsub__(self, other: Data) -> "Tensor":
        return Tensor.data_gate(other) + (-self)

    def __isub__(self, other: Data) -> "Tensor":
        self.data = self.data - Tensor.build_ndarray(other)
        return self
    
    @staticmethod
    def mul(a: "Tensor", b: "Tensor") -> "Tensor":
        output = a.data * b.data

        requires_grad = a.requires_grad or b.requires_grad
        dependencies = []

        def _backward(a: "Tensor", b: "Tensor"):
            def _bkwd(grad: np.ndarray) -> np.ndarray:
                grad = grad * b
                return Tensor.bkwd_broadcast(a)(grad)
            return _bkwd

        if a.requires_grad:
            dependencies.append(
                Leaf(
                    value=a,
                    grad_fn=_backward(a, b)
                )
            )

        if b.requires_grad:
            dependencies.append(
                Leaf(
                    value=b,
                    grad_fn=_backward(b, a)
                )
            )

        return Tensor(output, requires_grad, dependencies)
    
    def __mul__(self, other: Data) -> "Tensor":
        return Tensor.mul(self, Tensor.data_gate(other))
    
    def __rmul__(self, other: Data) -> "Tensor":
        return Tensor.mul(Tensor.data_gate(other), self)
    
    def __imul__(self, other: Data) -> "Tensor":
        self.data = self.data * Tensor.build_ndarray(other)
        return self

In [ ]:
a = Tensor([[1, 2],
            [3, 4]], requires_grad=True)
b = Tensor([10], requires_grad=True)

c = a + b

d = c - a * b + c * a * b

d.backward(np.ones_like(d.data))

a.grad, b.grad


(array([[Tensor([[111. 131.]
          [151. 171.]], requires_grad=True, shape=(2, 2)),
         Tensor([[111. 131.]
          [151. 171.]], requires_grad=True, shape=(2, 2))],
        [Tensor([[111. 131.]
          [151. 171.]], requires_grad=True, shape=(2, 2)),
         Tensor([[111. 131.]
          [151. 171.]], requires_grad=True, shape=(2, 2))]], dtype=object),
 array([Tensor([[ 84. 172.]
         [268. 372.]], requires_grad=True, shape=(2, 2))], dtype=object))

## Chain Rule

$$
\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}
$$

If we have a function composition:  

$$
f(x) = g(h(x))
$$

Then, by the chain rule:

$$
f'(x) = g'(h(x)) \cdot h'(x)
$$

In [34]:
t = Tensor([1, 2, 3], requires_grad=True)
t.data = [[1, 3, 5], [2, 3, 4]]
t_T = t.T

t_T.backward(np.ones_like(t_T.data))

In [35]:
InitMethod = Literal["xavier", "he", "he_leaky", "normal", "uniform"]


class Parameter(Tensor):
    def __init__(
        self,
        *shape: int,
        data: Optional[np.ndarray] = None,
        init_method: InitMethod = "xavier",
        gain: float = 1.0,
        alpha: float = 0.01,
    ):
        if data is None:
            data = self._init(shape, init_method, gain, alpha)

        super().__init__(data=data, requires_grad=True)

    def _init(
        self,
        shape: Tuple[int, ...], 
        init_method: InitMethod = "xavier", 
        gain: float = 1.0, 
        alpha: float = 0.01
    ):
        weights = np.random.randn(*shape)

        if init_method == "xavier":
            std = gain * np.sqrt(1.0 / shape[0])
            return std * weights
        if init_method == "he":
            std = gain * np.sqrt(2.0 / shape[0])
            return std * weights
        if init_method == "he_leaky":
            std = gain * np.sqrt(2.0 / (1 + alpha**2) * (1 / shape[0]))
            return std * weights
        if init_method == "normal":
            return gain * weights
        if init_method == "uniform":
            return gain * np.random.uniform(-1, 1, size=shape)

        raise ValueError(f"Unknown initialization method: {init_method}")

In [36]:
from typing import List

class Module:
    def __call__(self, *args, **kwds) -> Tensor:
        return self.forward(*args, **kwds)

    def forward(self, *args, **kwds):
        raise NotImplementedError()

    def parameters(self) -> List[Parameter]:
        r"""
        Returns a list of all parameters in the module and its submodules.
        """
        params = []
        for _, item in self.__dict__.items():
            if isinstance(item, Parameter):
                params.append(item)
            elif isinstance(item, Module):
                params.extend(item.parameters())
        return params

    def zero_grad(self) -> None:
        r"""
        Zeroes the gradients of all parameters in the module and its submodules.
        """
        for param in self.parameters():
            param.zero_grad()

    def params_count(self) -> int:
        return sum(param.size for param in self.parameters())


class Sequential(Module):
    def __init__(self, *modules: Module):
        self.modules = modules

    def parameters(self) -> List[Parameter]:
        r"""
        Returns a list of all parameters in the sequential module and its submodules.
        """
        params = []
        for module in self.modules:
            params.extend(module.parameters())
        return params

    def forward(self, x):
        r"""
        Passes the input through all modules in sequence.
        """
        for module in self.modules:
            input = module(*input)
        return input
    

class DummyModule(Module):
    def __init__(self, dims: int):
        super().__init__()

        self.dims = dims
        self.param = Parameter(dims)

    def forward(self, x: Tensor):
        return x.transpose()

In [37]:
model = Sequential(
    DummyModule(10),
    DummyModule(100),
    DummyModule(1000),
)

model.params_count()

1110

## Linear Layer: Matrix-Matrix Dot Product  

At layer $i$, the transformation is defined as:  

$$A_i(\mathbf{X}) = \mathbf{X} \mathbf{W}_i^T + \mathbf{B}_i$$

For a single layer:  

$$F_i(\mathbf{X}) = \sigma(A_i(\mathbf{X}))$$

where $A_i(\mathbf{X})$ is the linear transformation at layer $i$.  

A deep neural network applies these transformations layer by layer, leading to the final output:  

$$F(\mathbf{X}) = \sigma(A_L(\sigma(A_{L-1}(\dots \sigma(A_1(\mathbf{X})) \dots )))$$

Using **functional composition**, this process is compactly written as:  

$$F(\mathbf{X}) = A_L \circ \sigma \circ A_{L-1} \circ \dots \circ \sigma \circ A_1 (\mathbf{X})$$

In [38]:
class Linear(Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        bias: bool = True,
        init_method: InitMethod = "xavier",
    ):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features

        self.weights = Parameter(out_features, in_features, init_method=init_method)
        self.bias = Parameter(out_features, init_method="normal", gain=0.01) if bias else None

    def forward(self, x: Tensor):
        # Check dimensions of input tensors
        if x.ndim not in (2, 3):
            raise ValueError(f"Input must be 2D or 3D Tensor! x.ndim={x.ndim}")

        # Check if the last dimension of input matches in_features
        if x.shape[-1] != self.in_features:
            raise ValueError(
                f"Last dimension of input: {x.shape[-1]} does not match in_features: {self.in_features}"
            )

        # Compute matrix multiplication: x @ weight^T
        output = x @ self.weights.T

        if self.bias is not None:
            output = output + self.bias

        return output


## Dot product

**Backward Pass (Gradients Computation)**

Compute $\frac{\partial L}{\partial A}$ and $\frac{\partial L}{\partial B}$ using the chain rule.

**Gradient w.r.t. A**
The gradient of the loss $L$ with respect to $A$ is given by:

$$\frac{\partial L}{\partial A} = \frac{\partial L}{\partial Z} \times B^T$$

```python
if a.requires_grad:
    def _bkwd(grad: np.ndarray) -> np.ndarray:
        if b.ndim > 1:
            return grad @ b.data.swapaxes(-1, -2)  # grad * B^T
        return np.outer(grad, b.data.T).squeeze()  # Handles 1D case
```

- If $B$ is 2D, we use `b.data.swapaxes(-1, -2)` to compute $B^T$.
- If $B$ is 1D, we use `np.outer(grad, b.data.T)` to ensure correct shape.


**Gradient w.r.t. B**

The gradient of the loss $L$ with respect to $B$ is given by:

$$\frac{\partial L}{\partial B} = A^T \times \frac{\partial L}{\partial Z}$$

Where $A^T$ is the **transpose of A**.

This is implemented as:

```python
if b.requires_grad:
    def _bkwd(grad: np.ndarray) -> np.ndarray:
        if a.ndim > 1:
            return a.data.swapaxes(-1, -2) @ grad  # A^T * grad
        return np.outer(a.data.T, grad).squeeze()  # Handles 1D case
```

- If $A$ is 2D, we use `a.data.swapaxes(-1, -2)` to compute $A^T$.
- If $A$ is 1D, we use `np.outer(a.data.T, grad)`.


**Why Do We Use `swapaxes(-1, -2)` Instead of `.T`?**

`swapaxes(-1, -2)` is a **general approach** for transposing the last two dimensions. This ensures compatibility with **both 2D matrices and higher-dimensional tensors** (e.g., batches of matrices).

- `.T` works **only for 2D matrices**, affecting all axes in higher dimensions.
- `swapaxes(-1, -2)` **preserves batch and other leading dimensions**, modifying only the last two.

Example:

| Shape of Tensor | `.T` Output | `swapaxes(-1, -2)` Output |
|----------------|------------|---------------------------|
| `(m, n)` | `(n, m)` | `(n, m)` |
| `(batch, m, n)` | `(n, m, batch)` (incorrect) | `(batch, n, m)` (correct) |
| `(batch, time, m, n)` | `(n, m, time, batch)` (incorrect) | `(batch, time, n, m)` (correct) |


Matrix multiplication follows the chain rule. The backward pass computes gradients for both $A$ and $B$ using transposes. Uses `swapaxes(-1, -2)` to generalize for higher-dimensional cases.

| Tensor  | Gradient Formula | Code Implementation |
|---------|-----------------|----------------------|
| $A$ | $\frac{\partial L}{\partial A} = \frac{\partial L}{\partial Z} \times B^T$ | `grad @ b.data.swapaxes(-1, -2)` |
| $B$ | $\frac{\partial L}{\partial B} = A^T \times \frac{\partial L}{\partial Z}$ | `a.data.swapaxes(-1, -2) @ grad` |


In [39]:
def matmul(a: "Tensor", b: "Tensor") -> "Tensor":
    r"""
    Static method to perform matrix multiplication of two tensors.

    Args:
        a (Tensor): First matrix.
        b (Tensor): Second matrix.

    Returns:
        Tensor: Resulting tensor with tracked dependencies.
    """
    
    output = a.data @ b.data
    requires_grad = a.requires_grad or b.requires_grad
    dependencies = []

    if a.requires_grad:
        def _bkwd_a(grad: np.ndarray) -> np.ndarray:
            if b.ndim > 1:
                return grad @ b.data.swapaxes(-1, -2)
            return np.outer(grad, b.data.T).squeeze()
        
        dependencies.append(
            Leaf(
                value=a,
                grad_fn=_bkwd_a
            )
        )

    if b.requires_grad:
        def _bkwd_b(grad: np.ndarray) -> np.ndarray:
            if a.ndim > 1:
                return a.data.swapaxes(-1, -2) @ grad
            return np.outer(a.data.T, grad).squeeze()
        
        dependencies.append(
            Leaf(
                value=b,
                grad_fn=_bkwd_b
            )
        )

    return Tensor(output, requires_grad, dependencies)

## More operations: `bkwd_broadcasting`

In [40]:
import numpy as np

# Define array A with shape (3, 1)
A = np.array([
    [1],
    [2],
    [3],
])
print(f"Array A shape: {A.shape}")

# Define array B with shape (1, 4)
B = np.array([
    [1, 2, 3, 4],
])
print(f"Array B shape: {B.shape}")

# Perform broadcasting addition
result = A + B

print("A + B result: ")
print(result)

print(f"Result of A + B shape: {result.shape}")


Array A shape: (3, 1)
Array B shape: (1, 4)
A + B result: 
[[2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]]
Result of A + B shape: (3, 4)


The `bkwd_broadcast` method ensures gradients are correctly summed across broadcasted dimensions in `backward` mode

In [ ]:
def bkwd_broadcast(tensor: "Tensor"):
    def _bkwd(grad: np.ndarray) -> np.ndarray:
        if tensor.ndim == 0:
            return np.sum(grad)

        if grad.ndim == 0:
            return grad
        
        ndim_added = max(0, grad.ndim - tensor.ndim)

        if ndim_added > 0:
            grad = grad.sum(axis=tuple(range(ndim_added)), keepdims=False)

        reduce_axes = tuple(
            dim for dim in range(tensor.ndim)
            if tensor.shape[dim] == 1 and grad.shape[dim] > 1
        )

        if reduce_axes:
            grad = grad.sum(axis=reduce_axes, keepdims=True)

        if grad.shape != tensor.shape:
            grad = grad.reshape(tensor.shape)

        return grad

    return _bkwd

### Examples of backward broadcasting

In **Scenario 1**, `b` has shape `(1,)`, meaning it was **expanded to match both dimensions** of `a`. We **sum over all extra axes `(0,1)`** (`keepdims=False`) to return to shape `(1,)`.  


In [42]:
a = np.array([[1, 2], 
              [3, 4]])  # Shape: (2, 2)

b = np.array([10])      # Shape: (1,)  (Broadcasted across both axis)

c = a + b
print(f"c: {c}")

grad_c = np.ones_like(c)
print(f"grad_c: {grad_c}")

# Since `a` was not broadcasted, the gradient just passes through
grad_a = grad_c
print(f"grad_a: {grad_a}")

# Since `b` was **broadcasted along both axis**, we must **sum** over 
# that axis to reduce it back to `b`'s original shape `(1,)`
grad_b = grad_c.sum(axis=(0, 1), keepdims=False)
print(f"grad_b: {grad_b}")

c: [[11 12]
 [13 14]]
grad_c: [[1 1]
 [1 1]]
grad_a: [[1 1]
 [1 1]]
grad_b: 4



In **Scenario 2**, `b` has shape `(2,1)`, meaning it was broadcasted along axis `1` to match `a`'s shape `(2,2)`

We **sum over axis 1** (`keepdims=True`) to restore `b`'s original shape `(2,1)`.  

In [43]:
a = np.array([[1, 2], 
              [3, 4]])  # Shape: (2, 2)

b = np.array([[10],
              [20]])    # Shape: (2, 1)  (Broadcasted across axis 1)

c = a + b
print(f"c: {c}")

grad_c = np.ones_like(c)
print(f"grad_c: {grad_c}")

# Since `a` was not broadcasted, the gradient just passes through
grad_a = grad_c
print(f"grad_a: {grad_a}")

# Since `b` was **broadcasted along axis 1**, we must **sum** over 
# that axis to reduce it back to `b`'s original shape `(2,1)`
grad_b = grad_c.sum(axis=1, keepdims=True)
print(f"grad_b: {grad_b}")

c: [[11 12]
 [23 24]]
grad_c: [[1 1]
 [1 1]]
grad_a: [[1 1]
 [1 1]]
grad_b: [[2]
 [2]]


### `add`, `sub` and their friends

$$f(a, b) = a + b$$

The derivative of $a + b$ with respect to $a$ and $b$ is 1:

$$\frac{d}{da} (a + b) = 1$$

$$\frac{d}{db} (a + b) = 1$$

In [44]:
def add(a: "Tensor", b: "Tensor") -> "Tensor":
    output = a.data + b.data

    requires_grad = a.requires_grad or b.requires_grad

    dependencies = []

    if a.requires_grad:
        dependencies.append(
            Leaf(
                value=a,
                grad_fn=Tensor.bkwd_broadcast(a)
            )
        )

    if b.requires_grad:
        dependencies.append(
            Leaf(
                value=b,
                grad_fn=Tensor.bkwd_broadcast(b)
            )
        )

    return Tensor(output, requires_grad, dependencies)

### `mul`

The **multiplication** operation computes the element-wise product of two tensors.

$$f(a, b) = a \cdot b$$

The derivative of $a \cdot b$ with respect to $a$ and $b$ is:

$$\frac{d}{da} (a \cdot b) = b$$

$$\frac{d}{db} (a \cdot b) = a$$

In [ ]:
def mul(a: "Tensor", b: "Tensor") -> "Tensor":
    output = a.data * b.data

    requires_grad = a.requires_grad or b.requires_grad

    dependencies = []

    def _backward(a: "Tensor", b: "Tensor"):
        def _bkwd(grad: np.ndarray) -> np.ndarray:
            grad = grad * b
            return Tensor.bkwd_broadcast(a)(grad)
        
        return _bkwd

    if a.requires_grad:
        dependencies.append(
            Leaf(
                value=a,
                grad_fn=_backward(a, b)
            )
        )

    if b.requires_grad:
        dependencies.append(
            Leaf(
                value=b,
                grad_fn=_backward(b, a)
            )
        )

    return Tensor(output, requires_grad, dependencies)